В этом проекте я обучаю модель, которая будет принимать на вход текст вопроса и выдавать ответ. Модель на основе RNN(GRU) и с применением Attention.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

Данные - это датасет mail.ru: https://www.kaggle.com/c/chit-chat-encoders/data. Данные находятся в формате tsv, где табуляцией отделен вопрос от ответа. Я предварительно обрезала датасет до 500 тысяч примеров.

In [2]:
corpus_name = "mail_ru"
corpus = ''
datafile = 'train_cut_100000.txt'

Наша следующая задача — создать словарь и загрузить в память пары предложений «запрос/ответ».

Для операций с предложениями мы должны представить их в виде эмбедингов. Для этого мы определяем класс Voc, который хранит отображение слов в индексы, обратное отображение индексов в слова, количество каждого слова и общее количество слов. Класс содержит методы для добавления слова в словарь (addWord), добавления всех слов в предложение (addSentence) и для обрезки редко встречающихся слов (trim).

In [3]:
PAD_token = 0  # пад-токен
SOS_token = 1  # токен начала предложения
EOS_token = 2  # токен конца предложения

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # метод удаляет редко встречающиеся слова из словаря
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # заново инициализируем словари
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

        for word in keep_words:
            self.addWord(word)

In [4]:
MAX_LENGTH = 25  # максимальная длина предложения (до куда паддить)

# переводим строку в Unicode в ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# привозим к нижнему регистру, обрезаем редкие слова, убираем не буквенные символы
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^а-яА-Я.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# помещаем предобработанные пары вопрос/ответ в список и возвращаем объект voc
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# функция возвращает True если оба предложения в паре короче порога MAX_LENGTH
def filterPair(p):
    # для вопросов необходимо зарезервировать последнее место для токена EOS
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# фильтруем пары согласно функции выше
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# функция возвращает заполненный объект voc object и список пар
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

# загружаем объект voc и пары
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# печать пар, чтобы понять, как они выглядят после обработки
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 100001 sentence pairs
Trimmed to 100000 sentence pairs
Counting words...
Counted words: 109692

pairs:
['он трогает меня там и я не могу сказать нет почему так ?', 'хочется тебе этого . поэтому и молчишь . молчание знак согласия .']
['а приличная поза .это какая ?', 'все приличные . лишь бы поглубже забратся !']
['с какои фразы начинается ваше доброе утро . доброго утра .', 'в даныи момент рота подъем .']
['нужен ли автомобиль семье с маленьким ребенком ?', 'еще как нужен !']
['как вам мое фото ?', 'наверное подслушивать очень любишь .']
['ты в кого тут влюбилась ся признаваися ?', 'а не скажу .']
['что вас больше всего раздрожает в людях ?', 'зависть лицемерие грубость и чрезмерная раздражительность']
['почему девушки не любят добрых парнеи ?', 'а может они просто не замечают этого качества !']
['мужчина ! . что тебя может глубоко тронуть в женщине ?', 'ои да не пугаи ты их так']
['а ты знаешь зачем тебе даны чувства ?', 'чтобы т

Нам нужно преобразовать слова в наших парах предложений в соответствующие индексы из словаря и передать это моделям. Мы будем производить обучение мини-батчами. Чтобы разместить предложения разного размера в одном и том же батче, мы создадим наш входной тензор размером (max_length, batch_size), где предложения короче max_length дополняются нулями после EOS_token.
Если мы просто преобразуем наши предложения в тензоры, преобразовывая слова в их индексы (indexesFromSentence) и пады, наш тензор будет иметь размер (batch_size, max_length), и индексация по первому измерению вернет полную последовательность во всех таймстемпах. Однако нам нужно иметь возможность индексировать наш батч во времени и по всем последовательностям в батче. Поэтому мы транспонируем форму входного пакета в (max_length, batch_size), чтобы индексирование по первому измерению возвращало таймстемп для всех предложений в пакете. Мы обрабатываем это транспонирование неявно в функции zeroPadding.

Функция inputVar обрабатывает процесс преобразования предложений в тензор, в конечном итоге создавая тензор правильной формы, дополненный нулями. Она также возвращает тензор длин для каждой из последовательностей в пакете, который позже будет передан нашему декодеру.

Функция outputVar выполняет ту же функцию, что и inputVar, но вместо возврата тензора длин она возвращает тензор бинарной маски и максимальную целевую длину предложения. Тензор бинарной маски имеет ту же форму, что и выходной целевой тензор, но каждый элемент, являющийся PAD_token, равен 0, а все остальные — 1.

batch2TrainData просто берет набор пар и возвращает входные и целевые тензоры, используя вышеупомянутые функции.

In [5]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# возращает входную последовательность с падами и длины предложений
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# возращает целевую последовательность с падами, padding mask и максимальную длину целевого предложения
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# возвращает все эти величины для всех предложений в батче
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# вывод примера
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  230,    73,    13,   394,    73],
        [  267,   185,   126,  1113,   913],
        [  147,  4484,  2375,  1417,  3702],
        [ 2286,   106, 20710, 14050,  7183],
        [    9,  2311,  1288,   500,    15],
        [23721,     7,  6839,  5095,     2],
        [ 2528,  3392,    15,    15,     0],
        [  367,    15,     2,     2,     0],
        [   89,     2,     0,     0,     0],
        [23721,     0,     0,     0,     0],
        [ 2096,     0,     0,     0,     0],
        [   15,     0,     0,     0,     0],
        [    2,     0,     0,     0,     0]])
lengths: tensor([13,  9,  8,  8,  6])
target_variable: tensor([[ 2316, 66552,  1235,  8105,     9],
        [    9, 15734, 48674,    64,    73],
        [10607,    19,   278,  1518,    25],
        [   19,   131,     9,     2,   111],
        [   20, 66553,  3719,     0,  5016],
        [  276,    57,  1694,     0,  3150],
        [ 6441,    19,    19,     0,     2],
        [    2,     2,     2

В качестве модели используется RNN, а именно, bidirectional GRU.

In [6]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # преобразуем индексы слов в эмбединги
        embedded = self.embedding(input_seq)
        # формируем батч последовательностей с падами для модуля RNN
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        return outputs, hidden

In [7]:
# добавляем слой Attention согласно тому, как он описан в этой статье: https://arxiv.org/abs/1508.04025
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):

        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        attn_energies = attn_energies.t()

        return F.softmax(attn_energies, dim=1).unsqueeze(1)

Реализуем декодер.

In [8]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # определяем слои
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)

        rnn_output, hidden = self.gru(embedded, last_hidden)

        attn_weights = self.attn(rnn_output, encoder_outputs)

        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))

        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))

        output = self.out(concat_output)
        output = F.softmax(output, dim=1)

        return output, hidden

Поскольку мы имеем дело с батчами последовательностей, дополненных падами, мы не можем просто учитывать все элементы тензора при вычислении лосса. Мы определяем maskNLLLoss для расчета наших потерь на основе выходного тензора нашего декодера, целевого тензора и бинарного тензора маски, описывающего заполнение целевого тензора. Эта функция потерь вычисляет среднее отрицательное логарифмическое правдоподобие элементов, которые соответствуют 1 в маске тензора.

In [9]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

Функция обучения содержит алгоритм для одной итерации обучения (один батч входных данных).

Здесь используется teacher forcing. Это означает, что с некоторой вероятностью, заданной с помощью параметра Teacher_forcing_ratio, мы используем текущее целевое слово в качестве следующего ввода декодера, а не используем текущее предположение декодера. Этот метод помогает более эффективному обучению. Однако teacher forcing может привести к нестабильности модели во время вывода, поскольку у декодера может не быть достаточного шанса действительно создать свои собственные выходные последовательности во время обучения.
Второй прием — gradient clipping. Это широко используемый метод для борьбы с проблемой «взрывающегося градиента». По сути, обрезав или установив пороговое значение градиентов до максимального значения, мы предотвращаем экспоненциальный рост градиентов и либо слишком быстрый рост, либо слишком быстрое падение функции.

In [10]:
MAX_LENGTH = 20

def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):


    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()


    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    lengths = lengths.to("cpu")


    loss = 0
    print_losses = []
    n_totals = 0


    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)


    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    decoder_hidden = encoder_hidden[:decoder.n_layers]

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )

            decoder_input = target_variable[t].view(1, -1)

            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )

            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)

            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal


    loss.backward()


    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)


    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

Обучаем модель на наших данных. Функция trainIters отвечает за запуск n_iterations обучения с учетом переданных моделей, оптимизаторов, данных и т. д.

In [11]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # загружаем батчи для каждой итерации
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # инициализируем
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # цикл обучения
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]

        input_variable, lengths, target_variable, mask, max_target_len = training_batch


        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

 
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0


        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

Теперь нам необходимо установить, в каком виде отображать выходную последовательность. За это отвечает декодер. Жадное декодирование — это метод декодирования, который мы используем во время обучения, когда мы не используем teacher forcing. Другими словами, для каждого временного шага мы просто выбираем слово из decoder_output с самым высоким значением softmax. Этот метод декодирования оптимален на уровне одного временного шага.

Чтобы облегчить операцию жадного декодирования, мы определяем класс GreedySearchDecoder. При запуске объект этого класса принимает входную последовательность (input_seq) размера (input_seq length, 1), тензор входной длины (input_length) и max_length для ограничения длины предложения ответа.

In [12]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):

        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)

        decoder_hidden = encoder_hidden[:decoder.n_layers]

        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token

        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)

        for _ in range(max_length):

            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)

            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)

            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)

            decoder_input = torch.unsqueeze(decoder_input, 0)

        return all_tokens, all_scores

Ниже функция для визуальной оценки получившегося ответа.В этой функции мы получаем ответ с помощью модели, конвертируем индексы ответа в слова и возвращаем список декодированных слов.

AssessmentInput действует как пользовательский интерфейс для нашего чат-бота. При вызове появится текстовое поле ввода, в котором мы можем ввести наше предложение запроса. После ввода нашего входного предложения и нажатия Enter наш текст нормализуется так же, как и наши обучающие данные, и в конечном итоге передается в функцию оценки для получения декодированного выходного предложения. Мы зацикливаем этот процесс, поэтому мы можем продолжать общаться с нашим ботом, пока не введем либо «q», либо «quit».

Если вводится предложение, содержащее слово, которого нет в словаре, мы от лица модели просим пользователя перефразировать.

In [13]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):


    indexes_batch = [indexesFromSentence(voc, sentence)]

    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])

    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)

    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")

    tokens, scores = searcher(input_batch, lengths, max_length)

    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:

            input_sentence = input('> ')

            if input_sentence == 'q' or input_sentence == 'quit': break

            input_sentence = normalizeString(input_sentence)

            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)

            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Я тебя не понимаю, перефразируй, пожалуйста.")

Для запуска нам необходимо инициализировать отдельные модели энкодера и декодера. Мы устанавливаем желаемые конфигурации, выбираем запуск с нуля или устанавливаем контрольную точку для загрузки, а также строим и инициализируем модели.

In [14]:

model_name = 'cb_model'
attn_model = 'dot'

hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64


loadFilename = None
checkpoint_iter = 4000




if loadFilename:

    checkpoint = torch.load(loadFilename)

    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')

embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)

encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)

encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


Запускаем обучение. Сначала мы устанавливаем параметры обучения, затем инициализируем наши оптимизаторы и, наконец, вызываем функцию trainIters для запуска итераций обучения.
Я пробовала обучить модель сначала на 100 тысячах примеров, и получились неплохие результаты. Потом я взяла миллион примеров, и у меня не хватило памяти в колабе. Потом я взяла 500 тысяч примеров, но уменьшила число итераций, чтобы модель обучалась не дольше получаса. В итоге результаты были не очень хорошие, модель явно недообучилась. Поэтому я вернулась к 100 тысячам примеров, но вернула количество итераций равно 4000.

In [15]:

clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 100
save_every = 500


encoder.train()
decoder.train()


print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)


for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()


print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 100; Percent complete: 2.5%; Average loss: 7.8113
Iteration: 200; Percent complete: 5.0%; Average loss: 7.0917
Iteration: 300; Percent complete: 7.5%; Average loss: 6.9708
Iteration: 400; Percent complete: 10.0%; Average loss: 6.8943
Iteration: 500; Percent complete: 12.5%; Average loss: 6.8082
Iteration: 600; Percent complete: 15.0%; Average loss: 6.7702
Iteration: 700; Percent complete: 17.5%; Average loss: 6.7291
Iteration: 800; Percent complete: 20.0%; Average loss: 6.6326
Iteration: 900; Percent complete: 22.5%; Average loss: 6.6284
Iteration: 1000; Percent complete: 25.0%; Average loss: 6.5809
Iteration: 1100; Percent complete: 27.5%; Average loss: 6.5549
Iteration: 1200; Percent complete: 30.0%; Average loss: 6.5310
Iteration: 1300; Percent complete: 32.5%; Average loss: 6.4655
Iteration: 1400; Percent complete: 35.0%; Average loss: 6.4314
Iteration: 1500; Percent complete: 37.5%; Average loss: 6.

Производим визуальную оценку ответов модели.

In [16]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> как дела, моделька
Я тебя не понимаю, перефразируй, пожалуйста.
> как дела, мисс модель
Bot: никак . и не знаю . . . .
> ну вот. опять многоточия
Bot: я бы не знаю . . . . .
> такого же не было!
Bot: да и не сможешь ! ! ! ! ! ! ! !
> да я-то точно не смогу.
Bot: ты не обманешь ? . . . . . . . .
> нет. давай к вопросам.
Bot: не знаю . . как бы не знаю . . . . . .
> ты капризная. почему плохо работаешь?
Bot: я не знаю что я не знаю . . . .
> всё ты знаешь. почему трава зеленая?
Bot: я думаю что я не знаю . . я не знаю . . как то . .
> кто убил кеннеди?
Я тебя не понимаю, перефразируй, пожалуйста.
> кто президент америки?
Bot: я бы не знаю . . . . .
> почему солнце светит?
Bot: потому что у них не тянет . и тп
> почему люди стареют?
Bot: потому что они не бесчувственные
> как познакомились мистер и миссис смит?
Bot: не знаю что ли ? и наслождаися
> где живет катя?
Bot: в россии . . . . . .
> где находится санкт-петербург?
Bot: в россии . . . . . .
> где находится нью-йорк?
Bot: в палате

KeyboardInterrupt: ignored